# Library

In [99]:
#Library
import sys
import os
import os.path as pth

#!pip install torchplot
import torch
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import torch.backends.cudnn as cudnn
import torchplot as plt



import nets
import datasets
import tools
import layers as L
import train

from io import BytesIO
from datetime import datetime
from pytz import timezone
from slacker import Slacker
from quantization import *


import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import random
import itertools
import math

from tqdm import tqdm
from numba import jit

import warnings
warnings.simplefilter("ignore")

# Functions

In [103]:
# Custom functions - Completed

@jit
def amac(x, w):
    return (x * w).sum()


@jit
def relu(x):
    return np.maximum(0,x)


@jit
def padding(a):
    result = np.zeros((a.shape[0],a.shape[1]+2,a.shape[2]+2))
    for i in range(a.shape[0]):
        result[i] = np.pad(a[i],1)
    return result


@jit(nopython=True)
def maxpooling(x_original):
   
    depth = x_original.shape[0] 
    row = int((x_original.shape[1])/2)
    col = int((x_original.shape[2])/2)

    one_layer = np.zeros((depth,row,col))
    
    for d in range(depth):
        for i in range(row):
            for j in range(col):
                r = x_original[d,2*i:2*i+2,2*j:2*j+2].max()
                one_layer[d,i,j] = r
    return one_layer


@jit(nopython=True)
def quant_signed_05_np(original, bit=5):
    bit = bit -1
    original = np.clip(original, -0.9375, 0.9375)
    original = original * (2**bit)
    
    (row, col) = original.shape
    result = np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            result[i,j] = math.trunc(original[i,j])/ (2**bit)
    return result

@jit
def quant_signed_05_np_fc(original, bit=5):
    bit = bit -1
    original = np.clip(original, -0.9375, 0.9375)
    original = original * (2**bit)
    
    result = math.trunc(original)/ (2**bit)
    return result

@jit(nopython=True)
def quant_signed_15_np(original, bit=5):
    bit = bit -2
    original = np.clip(original, -1.875, 1.875)
    original = original * (2**bit)
    
    (row, col) = original.shape
    result = np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            result[i,j] = math.trunc(original[i,j])/ (2**bit)
    return result

@jit
def quant_signed_15_np_fc(original, bit=5):
    bit = bit -2
    original = np.clip(original,-1.875, 1.875)
    original = original * (2**bit)
    
    result = math.trunc(original)/ (2**bit)
    return result


get_bin = lambda x, n: format(x, 'b').zfill(n).replace("-","1")

# Partial sum function with point parameters - Completed

# This is partial sum function for convolution layer -> Matches with verilog
@jit(cache=True)
def partial_sum_fa_conv_point(original, bit=5, point = 1):
    a = original
    bit = bit - 2  + (point)
    value = 1.875/(2**(point))
    
    result = np.zeros(a.shape[0])
    for d in range(a.shape[0]):
        partial = a[d].sum()
        if partial > value:
            partial = value
        elif partial < -value:
            partial = -value
        else:
            partial = partial
        result[d] = math.trunc(partial* (2**bit)) / (2**bit)
    return result.sum()

# This function matches with our verilog model & reasonable accuracy
@jit(cache=True)
def partial_sum_fa_fc_point(original, bit=5, point=1):
    a = original
    bit = bit - 2  + (point)
    value = 1.875/(2**(point))
    
    partial = a.sum()
    if partial > value:
        partial = value
    elif partial < -value:
        partial = -value
    else:
        partial = partial
    result= math.trunc(partial* (2**bit)) / (2**bit)
    return result

# FC & Conv function with point parameters - completed
# These functions have same fixed point with bias & match with verilog file
@jit(cache=True)
def fc_fa_05(x_original, w, b, p=1):
   
    filt = w.shape[0]
    stage = int(x_original.shape[1]/8)
    c = np.zeros((1,filt))
    for f in range(filt):
        re = 0
        for i in range(stage):
            r = x_original[0,i*8:i*8+8] * w[f,i*8:i*8+8]
            re = re + partial_sum_fa_fc_point(r, point=p)
        c[0,f] = quant_signed_05_np_fc(re + b[f])
    return c

@jit(cache=True)
def fc_fa(x_original, w, b, p=1):
   
    filt = w.shape[0]
    stage = int(x_original.shape[1]/8)
    c = np.zeros((1,filt))
    for f in range(filt):
        re = 0
        for i in range(stage):
            r = x_original[0,i*8:i*8+8] * w[f,i*8:i*8+8]
            re = re + partial_sum_fa_fc_point(r, point=p)
        c[0,f] = quant_signed_15_np_fc(re + b[f])
    return c

@jit(cache=True)
def conv_custom_fa_05(x_original, w, b, p=1):
   
    filt = w.shape[0]
    depth = x_original.shape[0] 
    row = x_original.shape[1] - 2
    col = x_original.shape[2] - 2

    c = np.zeros((filt,row,col))
    one_layer = np.zeros((row,col))
    
    for f in range(filt):
        for i in range(row):
            for j in range(col):
                r = x_original[:,i:i+3,j:j+3] * w[f]
                one_layer[i,j] = partial_sum_fa_conv_point(r, point=p)
        c[f,:,:] = quant_signed_05_np(one_layer + b[f])
    return c


@jit(cache=True)
def conv_custom_fa(x_original, w, b, p=1):
   
    filt = w.shape[0]
    depth = x_original.shape[0] 
    row = x_original.shape[1] - 2
    col = x_original.shape[2] - 2

    c = np.zeros((filt,row,col))
    one_layer = np.zeros((row,col))
    
    for f in range(filt):
        for i in range(row):
            for j in range(col):
                r = x_original[:,i:i+3,j:j+3] * w[f]
                one_layer[i,j] = partial_sum_fa_conv_point(r, point=p)
        c[f,:,:] = quant_signed_15_np(one_layer + b[f])
    return c

In [127]:
# Function of saving parameter
def save_conv_weight(weight, bias, file_name, path,include_bias = 1):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        weight_name = file_name + "_weight.txt"
        bias_name = file_name + "_bias.txt"
    file = open(path + "/" + weight_name,'w')

    a = weight
    a = a * 16
    for d in range(a.shape[1]):
        for f in range(a.shape[0]):
            for i in range(3):
                for j in range(3):
                    file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
    file.close()

    file = open(path + "/" + bias_name,'w')
    a = bias
    a = a * 16
    for i in range(a.shape[0]):
        if include_bias:
            file.write(get_bin(int(a[i]),5)+"\n")
        else:
            file.write("0\n")
    file.close()
    print(f"Save {file_name} weight to {path}/{weight_name}\nSave {file_name} bias   to {path}/{bias_name}")

def save_fc_weight(weight, bias, file_name, path, include_bias = 1):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        file_name = file_name + ".txt"
    
    file = open(path + "/" + file_name,'w')

    weight = weight*16
    bias = bias * 16
    for w in range(0, weight.shape[0], 32):
        for mac in range(min(weight.shape[0] - w, 32)):
            if include_bias:
                file.write(get_bin(int(bias[w+mac]),5)+"\n")
            else:
                file.write("0\n")
        for z in range(0, weight.shape[1], 8):
            for mac in range(min(weight.shape[0] - w, 32)):
                for port in range(8):
                    file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")
                    
    file.close()
    print(f"Save {file_name[:-4]} weight to {path}/{file_name}")

def save_conv_result(x, file_name, path):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        output_name = file_name + "_output.txt"
    file = open(path + "/" + output_name,'w')

    save = x*16
    for d in range(x.shape[0]):
        for i in range(x.shape[1]):
            for j in range(x.shape[2]):
                file.write(get_bin(int(save[d,i,j]),10)+"\n")
    file.close()
    
def save_weight(model, nobias_file, bias_file):
    for conv in [model.conv1, model.conv2, model.conv3, model.conv4]:
        weight = conv.weight.detach().numpy()
#         weight = quant_signed_05(weight)
        print(weight.shape)
        weight = weight * 16
        bias = conv.bias.detach().numpy()
#         bias = quant_signed_05(bias)
        print(bias.shape)
        bias = bias * 16
        for w in range(0, weight.shape[0], 32):
            for mac in range(32):
                bias_file.write(get_bin(int(bias[w+mac]),5) +"\n")
                nobias_file.write("0\n")
            for z in range(weight.shape[1]):
                for mac in range(32):
                    for i in range(3):
                        for j in range(3):
                            bias_file.write(get_bin(int(weight[w+mac,z,i,j]),5)+"\n")
                            nobias_file.write(get_bin(int(weight[w+mac,z,i,j]),5)+"\n")
                            
    
    for fc in [model.fc5, model.fc6]:
        weight = fc.weight.detach().numpy()
#         weight = quant_signed_05(weight)
        print(weight.shape)
        weight = weight * 16
        bias = fc.bias.detach().numpy()
#         bias = quant_signed_05(bias)
        print(bias.shape)
        bias = bias * 16
        for w in range(0, weight.shape[0], 32):
            for mac in range(min(weight.shape[0] - w, 32)):
                bias_file.write(get_bin(int(bias[w+mac]),5) +"\n")
                nobias_file.write("0\n")
            for z in range(0, weight.shape[1], 8):
                for mac in range(min(weight.shape[0] - w, 32)):
                    for port in range(8):
                        bias_file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")
                        nobias_file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")

# Open model and file

In [105]:
# Load pretrained model and data

file = open('latest.txt' , 'r' )
line = file.readline()
pretrained_checkpoint = line
file.close()

# Default settings for arch, dataset, and checkpoint
arch = "CNN_627_large"
dataset = "cifar10"
batch_size = 256
pretrained_checkpoint = pretrained_checkpoint

# trainloader, _, testloader = datasets.get_mnist(batch_size)
model = nets.mnist_quant()

pretrained_ckpt = torch.load(pretrained_checkpoint)
model.load_state_dict(pretrained_ckpt['state_dict'])
print("########## Loaded checkpoint '{}'".format(pretrained_checkpoint))

print("mnist dataset with different fixed point")
with open("./data_quantized/quant_test_data_mnist.pkl","rb") as f:
    data_list = pickle.load(f)
with open("./data_quantized/quant_test_label_mnist.pkl","rb") as g:
    label_list = pickle.load(g)
    

#print("mnist dataset with same fixed point")
#with open("./data_quantized/quant_test_data_mnist_05.pkl","rb") as f:
#    data_list = pickle.load(f)
#with open("./data_quantized/quant_test_label_mnist_05.pkl","rb") as g:
#    label_list = pickle.load(g)
    
# Load Weights and biases

w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

#w1 = quant_signed_15(w1)
#w2 = quant_signed_15(w2)
#w3 = quant_signed_15(w3)
#w4 = quant_signed_15(w4)
#w5 = quant_signed_15(w5)
#w6 = quant_signed_15(w6)


w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()

Quant MNIST
########## Loaded checkpoint './checkpoints_quant/mnist_quant_mnist/2_18_Time_16_36/checkpoint_7_98.3.tar'
mnist dataset with different fixed point


In [116]:
file_no = open('./hope/weight_no_bias.txt','w')
file_bias = open('./hope/weight_bias.txt','w')
save_weight(model, file_no,file_bias)

file_no.close()
file_bias.close()

(32, 1, 3, 3)
(32,)
(64, 32, 3, 3)
(64,)
(128, 64, 3, 3)
(128,)
(256, 128, 3, 3)
(256,)
(32, 1024)
(32,)
(10, 32)
(10,)


In [86]:
path = "weight_new_wo_bias"

save_conv_weight(w1, b1, "conv1", path,include_bias=0)
save_conv_weight(w2, b2, "conv2", path,include_bias=0)
save_conv_weight(w3, b3, "conv3", path,include_bias=0)
save_conv_weight(w4, b4, "conv4", path,include_bias=0)
save_fc_weight(w5, b5, "fc5", path,include_bias=0)
save_fc_weight(w6, b6, "fc6", path,include_bias=0)

Save conv1 weight to weight_new_wo_bias/conv1_weight.txt
Save conv1 bias   to weight_new_wo_bias/conv1_bias.txt
Save conv2 weight to weight_new_wo_bias/conv2_weight.txt
Save conv2 bias   to weight_new_wo_bias/conv2_bias.txt
Save conv3 weight to weight_new_wo_bias/conv3_weight.txt
Save conv3 bias   to weight_new_wo_bias/conv3_bias.txt
Save conv4 weight to weight_new_wo_bias/conv4_weight.txt
Save conv4 bias   to weight_new_wo_bias/conv4_bias.txt
Save fc5 weight to weight_new_wo_bias/fc5.txt
Save fc6 weight to weight_new_wo_bias/fc6.txt


In [87]:
path = "weight_new_w_bias"

save_conv_weight(w1, b1, "conv1", path)
save_conv_weight(w2, b2, "conv2", path)
save_conv_weight(w3, b3, "conv3", path)
save_conv_weight(w4, b4, "conv4", path)
save_fc_weight(w5, b5, "fc5", path)
save_fc_weight(w6, b6, "fc6", path)

Save conv1 weight to weight_new_w_bias/conv1_weight.txt
Save conv1 bias   to weight_new_w_bias/conv1_bias.txt
Save conv2 weight to weight_new_w_bias/conv2_weight.txt
Save conv2 bias   to weight_new_w_bias/conv2_bias.txt
Save conv3 weight to weight_new_w_bias/conv3_weight.txt
Save conv3 bias   to weight_new_w_bias/conv3_bias.txt
Save conv4 weight to weight_new_w_bias/conv4_weight.txt
Save conv4 bias   to weight_new_w_bias/conv4_bias.txt
Save fc5 weight to weight_new_w_bias/fc5.txt
Save fc6 weight to weight_new_w_bias/fc6.txt


In [81]:
w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

# original
- bias   : _ . _ _ _ _

- input  : _ _ . _ _ _
- wegith : _ . _ _ _ _
- output : _ | _ _ . _ _ _  | _ _ _ _
- output : _ _ . _ _ _
 
- conv   : _ _ . _ _ _
- wegith : _ . _ _ _ _


# solution (problem) -> Solved
- bias   : _ . _ _ _ _
- wegith : _ . _ _ _ _
- input  : _ _ . _ _ _
- output : _ _ | _ . _ _ _ _ | _ _ _

- conv   : _ . _ _ _ _
- wegith : _ . _ _ _ _
- output : _ _ | _ . _ _ _ _ | _ _ _
- output : _ | _ _ . _ _ _ |_ _ _ _
-> Play with Accumulation layer

# solution2 
- bias   : _ . _ _ _ _
- wegith : _ _ . _ _ _
- input  : _ . _ _ _ _
- output : _ _ | _ . _ _ _ _ | _ _ _

- conv   : _ . _ _ _ _

# solution3 
- bias   : _ . _ _ _ _
- wegith : _ . _ _ _ _
- input  : _ . _ _ _ _
- output : _ | _ . _ _ _ _ | _ _ _ _
- conv   : _ . _ _ _ _



# potential solution
- bias   : _ . _ _ _ _
- wegith : _ . _ _ _ _
- input  : _ . _ _ _ _
- output : _ | _ . _ _ _ _ | _ _ _ _

- conv   : _ . _ _ _ _


# Comparing result

In [23]:
file = open('./result_new_wo_bias/conv1_input.txt','r')
file2 = open('./data_extracted/conv1_input_0317.txt','r')
for i,j in zip(file,file2):
    if i.replace('\n','') != j.replace('\n',''):
        print(i.replace('\n',''), j.replace('\n',''))

In [117]:
file = open('./result_new_wo_bias/conv1_output.txt','r')
file2 = open('./data_extracted/conv1_output_0317.txt','r')
for i,j in zip(file,file2):
    if i.replace('\n','') != j.replace('\n',''):
        pass#print(i.replace('\n',''), j.replace('\n',''))

In [ ]:
file = open('./result_new_wo_bias/pool_relu1_output.txt','r')
file2 = open('./data_extracted/pool_relu1_output_0317.txt','r')
for i,j in zip(file,file2):
    print(i.replace('\n',''), j.replace('\n',''))
    #if i.replace('\n','') != j.replace('\n',''):
        #print(i.replace('\n',''), j.replace('\n',''))

In [120]:
file = open('./result_new_wo_bias/conv2_output.txt','r')
file2 = open('./data_extracted/conv2_output_0317.txt','r')
for i,j in zip(file,file2):
    # print(i.replace('\n',''), j.replace('\n',''))
    if i.replace('\n','') != j.replace('\n',''):
        pass#print(i.replace('\n',''), j.replace('\n',''))

In [121]:
file = open('./result_new_wo_bias/conv3_output.txt','r')
file2 = open('./data_extracted/conv3_output_0317.txt','r')
num = 1
for i,j in zip(file,file2):
    #print(num, i.replace('\n',''), j.replace('\n',''))
    num += 1
    #if i.replace('\n','') != j.replace('\n',''):
    #    print(i.replace('\n',''), j.replace('\n',''))

In [123]:
file = open('./result_new_wo_bias/pool_relu1_output.txt','r')
file2 = open('./data_extracted/pool_relu1_output_0317.txt','r')
for i,j in zip(file,file2):
    pass#print(i.replace('\n',''), j.replace('\n',''))
    #if i.replace('\n','') != j.replace('\n',''):
        #print(i.replace('\n',''), j.replace('\n',''))

In [106]:
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

tensor(1024)


# Savine Result w/o bias hope

In [151]:

# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

x = x_original
x = x*8
print(x.shape)

file = open('./result_new_wo_bias/conv1_input.txt','w')
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

b1 = np.zeros(32)
x = x_original
x = padding(x)
x = conv_custom_fa(x,w1,b1, p=1)


file = open('./result_new_wo_bias/conv1_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()


x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu1_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b2 = np.zeros(64)
x = padding(x)
x = conv_custom_fa(x,w2,b2, p=1)

file = open('./result_new_wo_bias/conv2_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu2_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b3 = np.zeros(128)
x = padding(x)
x = conv_custom_fa(x,w3,b3, p=1)

file = open('./result_new_wo_bias/conv3_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu3_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b4 = np.zeros(256)
x = padding(x)
x = conv_custom_fa(x,w4,b4, p=1)

file = open('./result_new_wo_bias/conv4_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu4_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b5 = np.zeros(32)

x = fc_fa(x,w5,b5, p=1)

file = open('./result_new_wo_bias/fc5_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()


x = relu(x)
file = open('./result_new_wo_bias/relu5_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

print(x.shape)
b6 = np.zeros(10)

x = fc_fa(x,w6,b6, p=1)

file = open('./result_new_wo_bias/fc6_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_wo_bias/relu6_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x.shape)

tensor(1024)
torch.Size([1, 32, 32])
(32, 16, 16)
(64, 8, 8)
(128, 4, 4)
(256, 2, 2)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]
(1, 32)
(1, 10)


# Savie result with bias hope

In [135]:
w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias


w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()



file = open('./result_new_w_bias/conv1_input.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

x = x_original
x = x*8
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

x = x_original
x = padding(x)
x = conv_custom_fa(x,w1,b1, p=1)


file = open('./result_new_w_bias/conv1_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()


x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu1_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa(x,w2,b2, p=1)

file = open('./result_new_w_bias/conv2_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu2_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa(x,w3,b3, p=1)

file = open('./result_new_w_bias/conv3_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu3_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa(x,w4,b4, p=1)

file = open('./result_new_w_bias/conv4_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu4_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)

x = fc_fa(x,w5,b5, p=1)

file = open('./result_new_w_bias/fc5_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()


x = relu(x)
file = open('./result_new_w_bias/relu5_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

print(x.shape)

x = fc_fa(x,w6,b6, p=1)

file = open('./result_new_w_bias/fc6_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_w_bias/relu6_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x.shape)

tensor(1024)
torch.Size([1, 32, 32])
(32, 16, 16)
(64, 8, 8)
(128, 4, 4)
(256, 2, 2)
(1, 32)
(1, 10)


# Saving Result without bias

In [94]:
file = open('./result_new_wo_bias/conv1_input.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

x = x_original
x = x*8
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

b1 = np.zeros(32)
x = x_original
x = padding(x)
x = conv_custom_fa_05(x,w1,b1, p=1)


file = open('./result_new_wo_bias/conv1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()


x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b2 = np.zeros(64)
x = padding(x)
x = conv_custom_fa_05(x,w2,b2, p=1)

file = open('./result_new_wo_bias/conv2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b3 = np.zeros(128)
x = padding(x)
x = conv_custom_fa_05(x,w3,b3, p=1)

file = open('./result_new_wo_bias/conv3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b4 = np.zeros(256)
x = padding(x)
x = conv_custom_fa_05(x,w4,b4, p=1)

file = open('./result_new_wo_bias/conv4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b5 = np.zeros(32)

x = fc_fa_05(x,w5,b5, p=1)

file = open('./result_new_wo_bias/fc5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()


x = relu(x)
file = open('./result_new_wo_bias/relu5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

print(x.shape)
b6 = np.zeros(10)

x = fc_fa_05(x,w6,b6, p=1)

file = open('./result_new_wo_bias/fc6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_wo_bias/relu6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x.shape)

tensor(1024)
torch.Size([1, 32, 32])
(32, 16, 16)
(64, 8, 8)
(128, 4, 4)
(256, 2, 2)
(1, 32)
(1, 10)


# Saving result with bias

In [95]:
w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

w1 = quant_signed_15(w1)
w2 = quant_signed_15(w2)
w3 = quant_signed_15(w3)
w4 = quant_signed_15(w4)
w5 = quant_signed_15(w5)
w6 = quant_signed_15(w6)

w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()



file = open('./result_new_w_bias/conv1_input.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_05(x_original)).sum())
x_original = quant_signed_05(x_original)

x = x_original
x = x*16
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

x = x_original
x = padding(x)
x = conv_custom_fa_05(x,w1,b1, p=1)


file = open('./result_new_w_bias/conv1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()


x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa_05(x,w2,b2, p=1)

file = open('./result_new_w_bias/conv2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa_05(x,w3,b3, p=1)

file = open('./result_new_w_bias/conv3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa_05(x,w4,b4, p=1)

file = open('./result_new_w_bias/conv4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)

x = fc_fa_05(x,w5,b5, p=1)

file = open('./result_new_w_bias/fc5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()


x = relu(x)
file = open('./result_new_w_bias/relu5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

print(x.shape)

x = fc_fa_05(x,w6,b6, p=1)

file = open('./result_new_w_bias/fc6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_w_bias/relu6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x.shape)

tensor(1024)
torch.Size([1, 32, 32])
(32, 16, 16)
(64, 8, 8)
(128, 4, 4)
(256, 2, 2)
(1, 32)
(1, 10)


# Testing

In [133]:
# Evaluate with Custom model
correct = 0
#randlist = [i for i in range(50)]
#randlist = random.sample(range(0, 10000), 10)
randlist = [i for i in range(256)]
for i in randlist:
    y = label_list[i]
    x = data_list[i].view(1,32,32).numpy()

    x = padding(x)
    x = conv_custom_fa(x,w1,b1, p=1)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa(x,w2,b2, p=3)#3
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa(x,w3,b3, p=4)#4
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa(x,w4,b4, p=4)#4
    x = maxpooling(x)
    x = relu(x)

    x = x.reshape(1,1024)

    x = fc_fa(x,w5,b5, p=4)#4
    x = relu(x)

    x = fc_fa(x,w6,b6, p=1)#1


    if (np.argmax(x) == y.item()):
        correct += 1

print(correct/len(randlist))

0.8203125


In [147]:
x_original = data_list[0].view(1,32,32)
x = data_list[i].view(1,32,32).numpy()
print(x_original == x)

False


In [156]:
b1 = np.zeros(32)
b2 = np.zeros(64)
b3 = np.zeros(128)
b4 = np.zeros(256)
b5 = np.zeros(32)
b6 = np.zeros(10)

i=0
y = label_list[i]
x_original = data_list[0].view(1,32,32)
x = data_list[i].view(1,32,32)
x1 = quant_signed_15(x_original)
print(x==x1)

x = padding(x)
x = conv_custom_fa(x,w1,b1, p=1)

file = open('./result_new_wo_bias/conv1_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa(x,w2,b2, p=1)#3

file = open('./result_new_wo_bias/conv2_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()
x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa(x,w3,b3, p=1)#4

file = open('./result_new_wo_bias/conv3_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()
x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa(x,w4,b4, p=1)#4

file = open('./result_new_wo_bias/conv4_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()
x = maxpooling(x)
x = relu(x)
x = x.reshape(1,1024)

x = fc_fa(x,w5,b5, p=1)#4
file = open('./result_new_wo_bias/fc5_output.txt','w')
save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_wo_bias/relu5_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x)
x = fc_fa(x,w6,b6, p=1)#1
file = open('./result_new_wo_bias/fc6_output.txt','w')
save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)
file = open('./result_new_wo_bias/relu6_output.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])
[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.  ]]
